In [325]:
import pandas as pd
import numpy as np
import os
import math

In [326]:
def apply_pbc(value):
    if abs(value) > 0.5:
        return 1 - abs(value)
    return value


def mic_eucledian_distance(coor1, coor2):
    x_coor1, y_coor1, z_coor1 = coor1
    x_coor2, y_coor2, z_coor2 = coor2
    
    delta_x = x_coor1 - x_coor2
    delta_y = y_coor1 - y_coor2
    delta_z = z_coor1 - z_coor2

    distance = math.sqrt(sum([(apply_pbc(delta_x))**2, (apply_pbc(delta_y))**2, (apply_pbc(delta_z))**2]))
    return distance

def get_duplicate_closest24_w_data(dict):
    duplicate_closest24 = {}
    for coor120, values in dict.items():
        for entry in values:
            closest24 = entry["closest24"]
            dist = entry["dist"]

        if closest24 in duplicate_closest24:
            duplicate_closest24[closest24].append({"coor120": coor120, "dist": dist})
        else:
            duplicate_closest24[closest24] = [{"coor120": coor120, "dist": dist}]

    duplicate_closest24_w_data = {}
    for closest24, coor120s_dists in duplicate_closest24.items():
        if len(coor120s_dists) > 1:
            duplicate_closest24_w_data[f"Duplicate closest24: {closest24}"] = [{"coor120s and dists": coor120s_dists}]

    return duplicate_closest24_w_data

def get_atom_mapping_el_w_dist_closestduplicate(dict):
    filtered_data = {}
    for coor120, values in dict.items():
        for entry in values:
            closest24 = entry["closest24"]
            dist = entry["dist"]
            
        if closest24 in filtered_data:
            if dist < filtered_data[closest24]["dist"]:
                filtered_data[closest24] = {"coor120": coor120, "dist": dist}
        else:
            filtered_data[closest24] = {"coor120": coor120, "dist": dist}

    atom_mapping_el_w_dist_closestduplicate = {entry["coor120"]: {"closest24": key, "dist": entry["dist"]} for key, entry in filtered_data.items()}
    return atom_mapping_el_w_dist_closestduplicate

In [327]:
file_path_init = "7_2_CONTCAR_init_remake"
file_path = "1_2_CONTCAR_remake"

In [328]:
poscar_line_nr_start = 8        # index from 0
poscar_line_nr_end = 60
amount_Li = 25
n_decimal = 16

In [329]:
with open(file_path_init, 'r') as file_init:
    lines_init = file_init.readlines()
data_init = lines_init[poscar_line_nr_start:poscar_line_nr_start+amount_Li-1]

df_init = pd.DataFrame([string.strip().split() for string in data_init])
df_float_init = df_init.astype(float)

check_negative_init = df_float_init.lt(0).any().any()

if check_negative_init == True:
    breaksys.exit()    

coordinate_init = [row.to_numpy() for index, row in df_float_init.iterrows()]

In [330]:
data_init

['  0.9583519177297942  0.3168587380573808  0.3310391695362080\n',
 '  0.9943077422650554  0.7014125238969919  0.6961492118449879\n',
 '  0.0285849546705830  0.2757325394423627  0.6838661942695550\n',
 '  0.0731773730080812  0.7875552700544789  0.0790869644534405\n',
 '  0.1817815183141048  0.1900679353891917  0.9741949097442956\n',
 '  0.3447233333218262  0.6487857534945276  0.0476361236171942\n',
 '  0.7958971717513131  0.2151067750928907  0.0202511118463375\n',
 '  0.6948312525956580  0.6782724818628370  0.9674764056681765\n',
 '  0.1995770681557207  0.9885209462851521  0.2248956568588323\n',
 '  0.3219606260396715  0.0261483235156420  0.7304060430509630\n',
 '  0.8319874632018238  0.9616607440871594  0.8257118281602265\n',
 '  0.8247075716857841  0.0205261465698094  0.2147476471427280\n',
 '  0.2749091830435357  0.8002258307168023  0.4820060624121818\n',
 '  0.3296436573398320  0.2152861291607241  0.5256771562825286\n',
 '  0.7275120090899933  0.8224284960817591  0.5238865623585298

In [331]:
with open(file_path, 'r') as file:
    lines = file.readlines()
data = lines[poscar_line_nr_start:poscar_line_nr_start+amount_Li-1]
data_non_Li = lines[poscar_line_nr_start+amount_Li-1:poscar_line_nr_end]

df = pd.DataFrame([string.strip().split() for string in data])
df_float = df.astype(float)
df_non_Li = pd.DataFrame([string.strip().split() for string in data_non_Li])
df_float_non_Li = df_non_Li.astype(float)

check_negative = df_float.lt(0).any().any()

if check_negative == True:
    breaksys.exit()

coordinate = [row.to_numpy() for index, row in df_float.iterrows()]

In [332]:
data

['  0.9660441543982610  0.3283367829630452  0.3304881657415147\n',
 '  0.9912961640487372  0.7866388388604441  0.8083328523902946\n',
 '  0.0284562953814500  0.2009625187099880  0.8209024411843219\n',
 '  0.0208759304880026  0.6999420232859563  0.2826071187348129\n',
 '  0.3127535481871486  0.3245129564482546  0.9671485516820287\n',
 '  0.3135018411530068  0.6948142743143133  0.0138436059965953\n',
 '  0.6759378603652109  0.3137239557276421  0.0238637608651731\n',
 '  0.6658663363496352  0.6886257891390037  0.9561073325154819\n',
 '  0.1710569269062762  0.9668364475617172  0.1690830191903037\n',
 '  0.2184117480552486  0.0100443532307220  0.8104340716372637\n',
 '  0.8007559874845909  0.9783515569889948  0.7903474331695854\n',
 '  0.8236316685922560  0.0452098487626733  0.1628366413261168\n',
 '  0.2206534102193843  0.6975214104630874  0.4759451194048427\n',
 '  0.2897902014021367  0.1575774321426725  0.5258116632978397\n',
 '  0.8100248888116108  0.7141206018474680  0.5122521157531049

In [333]:
data_non_Li

['  0.5013437784957608  0.5005841300313585  0.5135142395967801\n',
 '  0.5062012924495933  0.9967821686619863  0.9978211364467555\n',
 '  0.9997182164381410  0.4969159360361462  0.0025339769273482\n',
 '  0.0056456619966022  0.0044935371653708  0.4913732617037816\n',
 '  0.1314687431970961  0.8818065551687827  0.6118143110792977\n',
 '  0.1178371831998687  0.1261228391134198  0.3586077318548823\n',
 '  0.8775563555635519  0.8916382929342045  0.3646500997315984\n',
 '  0.8939220633068671  0.1322348757827805  0.6195516881843121\n',
 '  0.8716962003917348  0.6313977703878489  0.1063602800423657\n',
 '  0.8729094155923182  0.3729655940003700  0.8857312644713439\n',
 '  0.1321405087234053  0.5992037116204909  0.8705573970649335\n',
 '  0.1197344162501527  0.3799149170391950  0.1321111782077781\n',
 '  0.6309757293586509  0.1238550031223131  0.8817290154242365\n',
 '  0.6284388196169670  0.8817109387268834  0.1260936813147704\n',
 '  0.3677798844566260  0.1151829135299319  0.1073918745984262

In [334]:
coor_final_reindex = []
atom_mapping_el_w_dist = {} 

for coor120 in coordinate:
    atom_mapping_w_dist_dict = {}
    atom_mapping_el_w_dist_closestduplicate = {}
    distance_prev = float("inf")
    closest24 = None

    for coor24 in coordinate_init:
        distance = mic_eucledian_distance(coor120, coor24)

        if distance < distance_prev:
            distance_prev = distance
            closest24 = coor24

    # coor_final_reindex.append(closest24)

    if closest24 is not None:
        atom_mapping_w_dist_dict['closest24'] = tuple(closest24)
        atom_mapping_w_dist_dict['dist'] = distance_prev
        
        if tuple(coor120) in atom_mapping_el_w_dist:
            new_entry = atom_mapping_el_w_dist[tuple(coor120)].copy()
            new_entry.append(atom_mapping_w_dist_dict)
            atom_mapping_el_w_dist[tuple(coor120)] = new_entry
        else:
            atom_mapping_el_w_dist[tuple(coor120)] = [atom_mapping_w_dist_dict.copy()]

duplicate_closest24_w_data = get_duplicate_closest24_w_data(atom_mapping_el_w_dist)

if len(duplicate_closest24_w_data) > 0:
    atom_mapping_el_w_dist_closestduplicate = get_atom_mapping_el_w_dist_closestduplicate(atom_mapping_el_w_dist)
    coor_reduced120_el_closestduplicate = [list(key) for key in atom_mapping_el_w_dist_closestduplicate.keys()]
else:
    atom_mapping_el_w_dist_closestduplicate = atom_mapping_el_w_dist.copy()
    # coor_reduced120_el_closestduplicate = coor_reduced120_el.copy()


In [335]:
duplicate_closest24_w_data

{'Duplicate closest24: (0.168633123097498, 0.548180069808847, 0.321652612435174)': [{'coor120s and dists': [{'coor120': (0.0208759304880026,
      0.6999420232859563,
      0.2826071187348129),
     'dist': 0.21537973226219023},
    {'coor120': (0.313689698450281, 0.5196731102275358, 0.2196373319437212),
     'dist': 0.17961395895474805}]}],
 'Duplicate closest24: (0.7275120090899933, 0.822428496081759, 0.5238865623585298)': [{'coor120s and dists': [{'coor120': (0.8100248888116108,
      0.714120601847468,
      0.5122521157531049),
     'dist': 0.13665407283076125},
    {'coor120': (0.5434311358162597, 0.8151899623957461, 0.657502127847896),
     'dist': 0.2275769839333209}]}]}

In [336]:
duplicate_closest24_w_data[f"Duplicate closest24: {(0.268633123097498, 0.2481800698088471, 0.221652612435174)}"] = [{"coor120s and dists": [
                {'coor120': (0.2208759304880026, 0.2999420232859563, 0.2826071187348129), 'dist': 0.21537973226219015},
                {'coor120': (0.213689698450281, 0.2196731102275358, 0.2196373319437212), 'dist': 0.27961395895474805}
            ]}]

In [337]:
duplicate_closest24_w_data

{'Duplicate closest24: (0.168633123097498, 0.548180069808847, 0.321652612435174)': [{'coor120s and dists': [{'coor120': (0.0208759304880026,
      0.6999420232859563,
      0.2826071187348129),
     'dist': 0.21537973226219023},
    {'coor120': (0.313689698450281, 0.5196731102275358, 0.2196373319437212),
     'dist': 0.17961395895474805}]}],
 'Duplicate closest24: (0.7275120090899933, 0.822428496081759, 0.5238865623585298)': [{'coor120s and dists': [{'coor120': (0.8100248888116108,
      0.714120601847468,
      0.5122521157531049),
     'dist': 0.13665407283076125},
    {'coor120': (0.5434311358162597, 0.8151899623957461, 0.657502127847896),
     'dist': 0.2275769839333209}]}],
 'Duplicate closest24: (0.268633123097498, 0.2481800698088471, 0.221652612435174)': [{'coor120s and dists': [{'coor120': (0.2208759304880026,
      0.2999420232859563,
      0.2826071187348129),
     'dist': 0.21537973226219015},
    {'coor120': (0.213689698450281, 0.2196731102275358, 0.2196373319437212),
     

In [355]:
duplicate_closest24_w_data

{'Duplicate closest24: (0.168633123097498, 0.548180069808847, 0.321652612435174)': [{'coor120s and dists': [{'coor120': (0.0208759304880026,
      0.6999420232859563,
      0.2826071187348129),
     'dist': 0.21537973226219023},
    {'coor120': (0.313689698450281, 0.5196731102275358, 0.2196373319437212),
     'dist': 0.17961395895474805}]}],
 'Duplicate closest24: (0.7275120090899933, 0.822428496081759, 0.5238865623585298)': [{'coor120s and dists': [{'coor120': (0.8100248888116108,
      0.714120601847468,
      0.5122521157531049),
     'dist': 0.13665407283076125},
    {'coor120': (0.5434311358162597, 0.8151899623957461, 0.657502127847896),
     'dist': 0.2275769839333209}]}],
 'Duplicate closest24: (0.268633123097498, 0.2481800698088471, 0.221652612435174)': [{'coor120s and dists': [{'coor120': (0.2208759304880026,
      0.2999420232859563,
      0.2826071187348129),
     'dist': 0.21537973226219015},
    {'coor120': (0.213689698450281, 0.2196731102275358, 0.2196373319437212),
     

In [356]:
for key, value in duplicate_closest24_w_data.items():
    # Extract the list of coor120s and dists
    coor120s_and_dists = value[0]['coor120s and dists']
    
    # Find the coor120 with the smallest distance
    min_distance_coor120 = max(coor120s_and_dists, key=lambda x: x['dist'])['coor120']

    print("Key:", key)
    print("coor120 with the smallest distance:", min_distance_coor120)
    print()

Key: Duplicate closest24: (0.168633123097498, 0.548180069808847, 0.321652612435174)
coor120 with the smallest distance: (0.0208759304880026, 0.6999420232859563, 0.2826071187348129)

Key: Duplicate closest24: (0.7275120090899933, 0.822428496081759, 0.5238865623585298)
coor120 with the smallest distance: (0.5434311358162597, 0.8151899623957461, 0.657502127847896)

Key: Duplicate closest24: (0.268633123097498, 0.2481800698088471, 0.221652612435174)
coor120 with the smallest distance: (0.213689698450281, 0.2196731102275358, 0.2196373319437212)



In [339]:
duplicate_closest24_w_data_mock = {
    'Duplicate closest24: (0.168633123097498, 0.5481800698088471, 0.321652612435174)': [
        {
            'coor120s and dists': [
                {'coor120': (0.0208759304880026, 0.6999420232859563, 0.2826071187348129), 'dist': 0.21537973226219015},
                {'coor120': (0.313689698450281, 0.5196731102275358, 0.2196373319437212), 'dist': 0.17961395895474805}
            ]
        }
    ],
    'Duplicate closest24: (0.268633123097498, 0.2481800698088471, 0.221652612435174)': [
        {
            'coor120s and dists': [
                {'coor120': (0.2208759304880026, 0.2999420232859563, 0.2826071187348129), 'dist': 0.21537973226219015},
                {'coor120': (0.213689698450281, 0.2196731102275358, 0.2196373319437212), 'dist': 0.27961395895474805}
            ]
        }
    ]    
}

In [340]:
duplicate_closest24_w_data_mock

{'Duplicate closest24: (0.168633123097498, 0.5481800698088471, 0.321652612435174)': [{'coor120s and dists': [{'coor120': (0.0208759304880026,
      0.6999420232859563,
      0.2826071187348129),
     'dist': 0.21537973226219015},
    {'coor120': (0.313689698450281, 0.5196731102275358, 0.2196373319437212),
     'dist': 0.17961395895474805}]}],
 'Duplicate closest24: (0.268633123097498, 0.2481800698088471, 0.221652612435174)': [{'coor120s and dists': [{'coor120': (0.2208759304880026,
      0.2999420232859563,
      0.2826071187348129),
     'dist': 0.21537973226219015},
    {'coor120': (0.213689698450281, 0.2196731102275358, 0.2196373319437212),
     'dist': 0.27961395895474805}]}]}

In [341]:
for key, value in duplicate_closest24_w_data_mock.items():
    # Extract the list of coor120s and dists
    coor120s_and_dists = value[0]['coor120s and dists']
    
    # Find the coor120 with the smallest distance
    min_distance_coor120 = min(coor120s_and_dists, key=lambda x: x['dist'])['coor120']

    print("Key:", key)
    print("coor120 with the smallest distance:", min_distance_coor120)
    print()

Key: Duplicate closest24: (0.168633123097498, 0.5481800698088471, 0.321652612435174)
coor120 with the smallest distance: (0.313689698450281, 0.5196731102275358, 0.2196373319437212)

Key: Duplicate closest24: (0.268633123097498, 0.2481800698088471, 0.221652612435174)
coor120 with the smallest distance: (0.2208759304880026, 0.2999420232859563, 0.2826071187348129)



In [342]:
coordinate

[array([0.96604415, 0.32833678, 0.33048817]),
 array([0.99129616, 0.78663884, 0.80833285]),
 array([0.0284563 , 0.20096252, 0.82090244]),
 array([0.02087593, 0.69994202, 0.28260712]),
 array([0.31275355, 0.32451296, 0.96714855]),
 array([0.31350184, 0.69481427, 0.01384361]),
 array([0.67593786, 0.31372396, 0.02386376]),
 array([0.66586634, 0.68862579, 0.95610733]),
 array([0.17105693, 0.96683645, 0.16908302]),
 array([0.21841175, 0.01004435, 0.81043407]),
 array([0.80075599, 0.97835156, 0.79034743]),
 array([0.82363167, 0.04520985, 0.16283664]),
 array([0.22065341, 0.69752141, 0.47594512]),
 array([0.2897902 , 0.15757743, 0.52581166]),
 array([0.81002489, 0.7141206 , 0.51225212]),
 array([0.70723126, 0.18418073, 0.47482973]),
 array([0.18607178, 0.4520853 , 0.66539395]),
 array([0.3136897 , 0.51967311, 0.21963733]),
 array([0.69928313, 0.49300521, 0.22709669]),
 array([0.83040247, 0.51989094, 0.69791027]),
 array([0.48647123, 0.2105332 , 0.71000547]),
 array([0.46378495, 0.84321144, 0.

In [343]:
# Extract the key and the list of coor120s and dists
key, coor120s_and_dists_list = duplicate_closest24_w_data_mock.popitem()

# Extract the list of coor120s and dists
coor120s_and_dists = coor120s_and_dists_list[0]['coor120s and dists']

# Find the coor120 with the smallest distance
min_distance_coor120 = min(coor120s_and_dists, key=lambda x: x['dist'])['coor120']

print("Key:", key)
print("coor120 with the smallest distance:", min_distance_coor120)

Key: Duplicate closest24: (0.268633123097498, 0.2481800698088471, 0.221652612435174)
coor120 with the smallest distance: (0.2208759304880026, 0.2999420232859563, 0.2826071187348129)


In [344]:
for i in duplicate_closest24_w_data_mock:
    print(i)

Duplicate closest24: (0.168633123097498, 0.5481800698088471, 0.321652612435174)


In [345]:
atom_mapping_el_w_dist

{(0.966044154398261,
  0.3283367829630452,
  0.3304881657415147): [{'closest24': (0.9583519177297942,
    0.3168587380573808,
    0.331039169536208),
   'dist': 0.013828218431957142}],
 (0.9912961640487372,
  0.7866388388604441,
  0.8083328523902946): [{'closest24': (0.9943077422650554,
    0.7014125238969919,
    0.6961492118449879),
   'dist': 0.14091757722726123}],
 (0.02845629538145,
  0.200962518709988,
  0.8209024411843219): [{'closest24': (0.028584954670583,
    0.2757325394423627,
    0.683866194269555),
   'dist': 0.15610735255591707}],
 (0.0208759304880026,
  0.6999420232859563,
  0.2826071187348129): [{'closest24': (0.168633123097498,
    0.548180069808847,
    0.321652612435174),
   'dist': 0.21537973226219023}],
 (0.3127535481871486,
  0.3245129564482546,
  0.9671485516820287): [{'closest24': (0.1817815183141048,
    0.1900679353891917,
    0.9741949097442956),
   'dist': 0.18782648231433988}],
 (0.3135018411530068,
  0.6948142743143133,
  0.0138436059965953): [{'closest24

In [346]:
type(coor_reduced120_el_closestduplicate)

list

In [347]:
coor_reduced120_el_closestduplicate

[[0.966044154398261, 0.3283367829630452, 0.3304881657415147],
 [0.9912961640487372, 0.7866388388604441, 0.8083328523902946],
 [0.02845629538145, 0.200962518709988, 0.8209024411843219],
 [0.313689698450281, 0.5196731102275358, 0.2196373319437212],
 [0.3127535481871486, 0.3245129564482546, 0.9671485516820287],
 [0.3135018411530068, 0.6948142743143133, 0.0138436059965953],
 [0.6759378603652109, 0.3137239557276421, 0.0238637608651731],
 [0.6658663363496352, 0.6886257891390037, 0.9561073325154819],
 [0.1710569269062762, 0.9668364475617172, 0.1690830191903037],
 [0.2184117480552486, 0.010044353230722, 0.8104340716372637],
 [0.8007559874845909, 0.9783515569889948, 0.7903474331695854],
 [0.823631668592256, 0.0452098487626733, 0.1628366413261168],
 [0.2206534102193843, 0.6975214104630874, 0.4759451194048427],
 [0.2897902014021367, 0.1575774321426725, 0.5258116632978397],
 [0.8100248888116108, 0.714120601847468, 0.5122521157531049],
 [0.7072312592123313, 0.1841807312656225, 0.4748297332366084],


In [348]:
atom_mapping_el_w_dist_closestduplicate

{(0.966044154398261,
  0.3283367829630452,
  0.3304881657415147): {'closest24': (0.9583519177297942,
   0.3168587380573808,
   0.331039169536208), 'dist': 0.013828218431957142},
 (0.9912961640487372,
  0.7866388388604441,
  0.8083328523902946): {'closest24': (0.9943077422650554,
   0.7014125238969919,
   0.6961492118449879), 'dist': 0.14091757722726123},
 (0.02845629538145,
  0.200962518709988,
  0.8209024411843219): {'closest24': (0.028584954670583,
   0.2757325394423627,
   0.683866194269555), 'dist': 0.15610735255591707},
 (0.313689698450281,
  0.5196731102275358,
  0.2196373319437212): {'closest24': (0.168633123097498,
   0.548180069808847,
   0.321652612435174), 'dist': 0.17961395895474805},
 (0.3127535481871486,
  0.3245129564482546,
  0.9671485516820287): {'closest24': (0.1817815183141048,
   0.1900679353891917,
   0.9741949097442956), 'dist': 0.18782648231433988},
 (0.3135018411530068,
  0.6948142743143133,
  0.0138436059965953): {'closest24': (0.3447233333218262,
   0.64878575

In [349]:
coor_final_reindex

[]

In [350]:
coordinate

[array([0.96604415, 0.32833678, 0.33048817]),
 array([0.99129616, 0.78663884, 0.80833285]),
 array([0.0284563 , 0.20096252, 0.82090244]),
 array([0.02087593, 0.69994202, 0.28260712]),
 array([0.31275355, 0.32451296, 0.96714855]),
 array([0.31350184, 0.69481427, 0.01384361]),
 array([0.67593786, 0.31372396, 0.02386376]),
 array([0.66586634, 0.68862579, 0.95610733]),
 array([0.17105693, 0.96683645, 0.16908302]),
 array([0.21841175, 0.01004435, 0.81043407]),
 array([0.80075599, 0.97835156, 0.79034743]),
 array([0.82363167, 0.04520985, 0.16283664]),
 array([0.22065341, 0.69752141, 0.47594512]),
 array([0.2897902 , 0.15757743, 0.52581166]),
 array([0.81002489, 0.7141206 , 0.51225212]),
 array([0.70723126, 0.18418073, 0.47482973]),
 array([0.18607178, 0.4520853 , 0.66539395]),
 array([0.3136897 , 0.51967311, 0.21963733]),
 array([0.69928313, 0.49300521, 0.22709669]),
 array([0.83040247, 0.51989094, 0.69791027]),
 array([0.48647123, 0.2105332 , 0.71000547]),
 array([0.46378495, 0.84321144, 0.

In [351]:
are_equal = all(np.array_equal(coor_reindex, coor) for coor_reindex, coor in zip(coor_reduced120_el_closestduplicate, coordinate))

if are_equal:
    print("The arrays are the same.")
else:
    print("The arrays are different.")
    indices_diff = [i for i, (reindex, coord) in enumerate(zip(coor_reduced120_el_closestduplicate, coordinate)) if not np.array_equal(reindex, coord)]
    print("Indices where the arrays differ:", indices_diff)

The arrays are different.
Indices where the arrays differ: [3, 17, 18, 19, 20, 21]


In [352]:
df_coordinate = pd.DataFrame(coordinate)
df_concat = pd.concat([df_coordinate, df_float_non_Li], ignore_index=True)

In [353]:
for i in range(df_concat.shape[0]):
    for j in range(df_concat.shape[1]):
        temp = df_concat[j][i]
        df_concat[j][i] = '{:.{width}f}'.format(float(temp), width=n_decimal)

/tmp/ipykernel_17541/1384378018.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat[j][i] = '{:.{width}f}'.format(float(temp), width=n_decimal)


In [354]:
row_list = df_concat.to_string(index=False, header=False).split('\n')
row_list_space = ['  '.join(string.split()) for string in row_list] # 2 spaces of distance
row_list_w_beginning = ['  ' + row for row in row_list_space]       # 2 spaces in the beginning
absolute_correct_list = '\n'.join(row_list_w_beginning).splitlines()        

line_append_list = []
for idx_c, line in enumerate(absolute_correct_list):
    line_new_line = str(line) + '\n'
    line_append_list.append(line_new_line)

file_list = lines[:poscar_line_nr_start] + line_append_list

poscar_filename = file_path+"_reindexed"
destination_path = poscar_filename

with open(destination_path, "w") as poscar_positive_file:
    for item in file_list:
        poscar_positive_file.writelines(item)

In [ ]:
import numpy as np

# Function to calculate distance between two 3D points
def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

# Function to reorder Li atoms in POSCAR_final based on proximity to POSCAR_init
def reorder_Li_atoms(init_positions, final_positions):
    # Extract Li atom positions from POSCAR_init and POSCAR_final
    init_Li_positions = [pos for pos, atom in zip(init_positions, init_types) if atom == 'Li']
    final_Li_positions = [pos for pos, atom in zip(final_positions, final_types) if atom == 'Li']

    # Calculate distances between each Li atom in POSCAR_init and POSCAR_final
    distances = []
    for init_Li_pos in init_Li_positions:
        distances.append([calculate_distance(init_Li_pos, final_Li_pos) for final_Li_pos in final_Li_positions])

    # Reorder Li atoms in POSCAR_final based on closest distances
    reordered_final_positions = []
    for dist_list in zip(*distances):
        min_index = np.argmin(dist_list)
        reordered_final_positions.append(final_Li_positions[min_index])

    return reordered_final_positions

# Example POSCAR_init and POSCAR_final (you can replace these with your actual data)
POSCAR_init = [
    'Li24 P4 S20 Cl4',
    '1.00000000000000',
    '9.6575578267957152    0.0000000000000000    0.0000000000000000',
    # ... (other lines)
    ' 0.8940071395675603  0.5892290157360063  0.8993668812576385'
]

POSCAR_final = [
    'Li24 P4 S20 Cl4',
    '1.00000000000000',
    '9.6593030851370756    0.0000000000000000    0.0000000000000000',
    # ... (other lines)
    ' 0.5180832595321223  0.7750078991287682  0.5704014333223363'
]

# Extract atom types and positions from POSCAR_init and POSCAR_final
init_types = POSCAR_init[5].split()
init_positions = [list(map(float, line.split()[:3])) for line in POSCAR_init[8:]]

final_types = POSCAR_final[5].split()
final_positions = [list(map(float, line.split()[:3])) for line in POSCAR_final[8:]]

# Reorder Li atoms in POSCAR_final based on proximity to POSCAR_init
reordered_final_positions = reorder_Li_atoms(init_positions, final_positions)

# Print the reordered POSCAR_final
print('\n'.join(POSCAR_final[:8] + ['  {:18.15f}  {:18.15f}  {:18.15f}'.format(*pos) for pos in reordered_final_positions]))
